In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

pd.options.display.mpl_style = 'default'
plt.rcParams['figure.figsize'] = (15, 5)


In [ ]:
import requests
import json

author_endpoint = "http://citation-analysis.appspot.com/author/"
publication_endpoint = "http://citation-analysis.appspot.com/publication/"
author = "aKeZJnUAAAAJ"

resp = requests.get(author_endpoint + author)
data = json.loads(resp.text)
publications = data["publication_ids"]

total = 0
for pub in publications:
    # print publication_endpoint + pub
    resp2 = requests.get(publication_endpoint + pub)
    try:
        data2 = json.loads(resp2.text)
    except:
        continue
    print unicode(data2["total_citations"]) +  u" --- " + data2["title"]
    print "=============="
    total += data2["total_citations"]
    
print total

In [ ]:
import requests
import json
import time


author_endpoint = "http://citation-analysis.appspot.com/author/"
tofetch_authors = set(["PA9La6oAAAAJ"])
fetched_authors = set([])

while len(tofetch_authors)>0:
    author = tofetch_authors.pop()
    print "Pending:", len(tofetch_authors)
    print "Done:", len(fetched_authors)
    print "Fetching:", author
    
    resp = requests.get(author_endpoint + author)
    try:
        data = json.loads(resp.text)
        fetched_authors.add(author)
    except:
        tofetch_authors.add(author)
        print "==========================="
        continue
    
    name = data["name"]
    print "Added ", name
    print "==========================="
    coauthors = data["coauthor_ids"]
    for coauthor in coauthors:
        if coauthor in fetched_authors or coauthor is None:
            continue
        else:
            tofetch_authors.add(coauthor)

   

In [ ]:
import requests
import json
import csv
import pandas as pd
import requests

author_endpoint = "http://citation-analysis.appspot.com/author/"
authors = ['-Km63D4AAAAJ', '-MQgcjYAAAAJ', '-V1HvPYAAAAJ', '0epc43IAAAAJ', '0hgmLKMAAAAJ', '2xBzrdYAAAAJ', '3BX3vWcAAAAJ', '5nUaWmMAAAAJ', '7OW6weoAAAAJ', '8JkiWl0AAAAJ', '8N_cfXsAAAAJ', '8_a-wgUAAAAJ', '8fztli4AAAAJ', '98Uq2IsAAAAJ', '9twfX9cAAAAJ', 'BxCCm4cAAAAJ', 'DKCx8hcAAAAJ', 'DSLquxEAAAAJ', 'D_JpQnAAAAAJ', 'E2uuNVoAAAAJ', 'EicYvbwAAAAJ', 'Fa13zIoAAAAJ', 'KC7FG9QAAAAJ', 'K__JmtcAAAAJ', 'KpNeomkAAAAJ', 'M0OB5XQAAAAJ', 'MDIyLnwAAAAJ', 'Md2HV1cAAAAJ', 'N2Rbbw8AAAAJ', 'P0YH9McAAAAJ', 'PA9La6oAAAAJ', 'Q3YPpnwAAAAJ', 'Se3D91QAAAAJ', 'U0NU_94AAAAJ', 'UMYf8QMAAAAJ', 'UWZA0v4AAAAJ', 'V4OPEAgAAAAJ', 'V6LJpwgAAAAJ', 'W3oUoyEAAAAJ', 'WGgk0_AAAAAJ', 'XmLMNyUAAAAJ', 'ZHMbRd4AAAAJ', 'ZasL8IoAAAAJ', 'bgfl9AYAAAAJ', 'blwKAkUAAAAJ', 'c1dydJIAAAAJ', 'f0Wc8tkAAAAJ', 'fP-OzuoAAAAJ', 'fyHJ87oAAAAJ', 'gXrMlhIAAAAJ', 'gvJX5l8AAAAJ', 'h0ZyN6QAAAAJ', 'jbB0DYUAAAAJ', 'k4Y1nEwAAAAJ', 'kGKlHp0AAAAJ', 'm50LZsUAAAAJ', 'mhmvCgsAAAAJ', 'nKR2tGvMLFMC', 'niwimDYAAAAJ', 'oQHsB5kAAAAJ', 'oj_DO_AAAAAJ', 'pju_kjIAAAAJ', 'v8n2lqAAAAAJ', 'vPEmzqYAAAAJ', 'wyCIJfQAAAAJ', 'yrFC-2EAAAAJ', 'zpN9P0sAAAAJ', 'zwax5qkAAAAJ']
headers = ['Scholar ID', 'Name', 'Year', 'Citations', 'Age', 'last_year_citations', 'last2_year_citations', 'growth_absolute', 'growth_relative']

tuples = []
for author in authors:
    resp = requests.get(author_endpoint + author)
    if resp.status_code != 200:
        continue
    data = json.loads(resp.text)
    name = data["name"]
    yearly_cits = data["citations_per_year"]
    start = min([int(y) for y in yearly_cits.keys()])

    for year in sorted(yearly_cits.keys()):
        current_year = int(year)

        citations = int(yearly_cits.get(year))
        if year==2015:
            day_of_year = datetime.now().timetuple().tm_yday
            citations = (365 * citations) / day_of_year
        
        last_year_citations = yearly_cits.get(str(current_year-1))
        if last_year_citations == None:
            last_year_citations = 0
        
        last2_year_citations = yearly_cits.get(str(current_year-2))
        if last2_year_citations == None:
            last2_year_citations = 0
        
        growth_absolute = last_year_citations-last2_year_citations
        growth_relative = growth_absolute/(last2_year_citations+1.0)
        
        t = (author, name, int(year), int(citations), int(year)-start, last_year_citations, last2_year_citations, growth_absolute, growth_relative)
        tuples.append(t)

df = pd.DataFrame(data=tuples, columns = headers)

In [ ]:
# Let's do a scatterplot 
df.plot(kind='scatter', x='last_year_citations', y='Citations')

In [ ]:
# Apparently, there is a strong autocorrelation between current year and last year citation. Let's 
# create a Growth variable for predicting growth of citations
df["Growth"] = map(lambda x,y: x-y, df["Citations"],  df["last_year_citations"])
# and relative growth...
df["RelGrowth"] = map(lambda x,y: x/y if y>0 else 1.0, df["Citations"],  df["last_year_citations"])

df.head()

In [ ]:
df.plot(kind='scatter', x='growth_absolute', y='Growth')

df.plot(kind='scatter', x='growth_relative', y='RelGrowth')

In [ ]:
# This creates a pivot table with Authors as columns, in order to do some exploratory plots
byyear = df.pivot(index='Age', columns='Name', values='Citations')
byyear.head()

In [ ]:
byyear.plot(legend=False, logy=False)
byyear.plot(legend=False, logy=True)
byyear.cumsum().plot(legend=False, logy=True)
byyear.cumsum().plot(legend=False, logy=False)

In [ ]:
import statsmodels.formula.api as sm

result = sm.ols(formula="Growth ~ growth_absolute", data=df).fit()
print result.summary()

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable, Independence, Autoregressive)
from statsmodels.genmod.families import Poisson

mod1 = GEE.from_formula("Citations ~ last_year_citations", 
                         "Name", df, cov_struct=Independence(), family=Poisson())
rslt1 = mod1.fit()
print rslt1.summary()

In [ ]:
df["constant"] = 1
from statsmodels.discrete.discrete_model import Poisson

df[['Citations', 'last_year_citations', 'growth_absolute', 'constant']] = df[['Citations', 'last_year_citations', 'growth_absolute', 'constant']].astype(float)

model = Poisson.from_formula("Citations ~ last_year_citations + growth_absolute + constant", df)
results = model.fit('newton')
print results.summary()
